In [83]:
import torch
import warnings
warnings.filterwarnings('ignore')
import re
import gensim.downloader as api
import pandas as pd
import os
import numpy as np
from sentence_transformers import SentenceTransformer, util
import random

In [2]:
model_w2v = api.load("word2vec-google-news-300")
#model_w2v.vocab

In [12]:
type (model_w2v)

gensim.models.keyedvectors.Word2VecKeyedVectors

In [36]:
results_dir = 'C:\\Users\\RonyArmon\\Projects_Code\\Cluster_Activities\\results'
names = open(os.path.join(results_dir, 'names.txt')).read().split('\n')
names = list(set(names))
#names = names[:100]
print('{} names'.format(len(names)))

17077 names


# Name vectors   
Mean pooling of token vectors 

In [78]:
punctuation_marks="=|\+|_|\.|:|\/|\*|\'|,|\?|-|\(|\)|{|}|\[|\]"
names_embeddings, names_not_in_vocab, embed_names = [], [], []
names = [n for n in names if n]
for index, name in enumerate(names):
#     print(50*'-')
#     print('name:',index, name)
    name = re.sub(punctuation_marks,' ', name)
    name = re.sub('\s{2,}', ' ', name)
    token_embeddings = []
    tokens = name.split(' ')
    tokens = [t for t in tokens if len(t)>1]
    not_in_vocab = []
    for token in tokens:
        if token in model_w2v.vocab:
            token_embedding = model_w2v[token]
            token_embeddings.append(token_embedding)
        else:
            not_in_vocab.append(token)
    token_embeddings = np.array(token_embeddings)
    if (len(token_embeddings))>0:
        name_embedding = token_embeddings.mean(axis=0)
        name_embedding = torch.tensor(name_embedding)
        names_embeddings.append(name_embedding)
        embed_names.append(name)
        names_not_in_vocab.append(not_in_vocab)
names_embeddings = torch.stack(names_embeddings)

In [79]:
print(type(names_embeddings), names_embeddings.shape)

<class 'torch.Tensor'> torch.Size([17069, 300])


In [84]:
sample = random.choices(range(len(names)), k=100)
results_df = pd.DataFrame()
#for name_index in range(len(names)):
for name_index in sample:
    query = names[name_index]
    name_embedding = names_embeddings[name_index]
    cosine_scores = util.cos_sim(name_embedding, names_embeddings)
    cosine_scores = list(cosine_scores[0])
    cosine_scores = [float(s) for s in cosine_scores]
    sorted_scores = cosine_scores.copy()
    sorted_scores.sort(reverse = True)
    top10_scores = sorted_scores[:10]
    top10_scores = [i for i in top10_scores if i>0.7]
    results = []
    for score in top10_scores: 
        score_index = cosine_scores.index(score)
        name = names[score_index]
        if score>0.9999: score = 'query'
        else: score = round(score, 2)
        results.append([name, score])
    result_df = pd.DataFrame(results, columns = ['name', 'score'])
    #print(50*'-')
    #print('query:', query)
    #if len (result_df)==0: print('no cluster')
    #else: display(result_df.style.set_properties(**{'text-align': 'left', 'colheader_justify':'left'}))
    if len (result_df)>1:
        results_df = results_df.append(result_df)

In [87]:
results_df.head(50)

,name,score
0,142a - landscape master plan revision,query
1,erection piping system (outside) 20pab20-spool6,0.9
2,termination of tp-f1a,0.9
3,termination of tp-f1a,0.9
4,termination of tp-f1a,0.9
5,termination of tp-f1a,0.9
6,termination of tp-f1a,0.9
7,termination of tp-f1a,0.9
8,termination of tp-f1a,0.9
9,termination of tp-f1a,0.9
